In [ ]:
import pandas as pd 

#Load and explore new csv data after preprocessing
final_df_train = pd.read_csv("final_train_data",  low_memory=False)
final_df_train.head()


,isic_id,target,patient_id,age_approx,sex,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,...,tbp_lv_location_simple_Torso Front,tbp_lv_location_simple_Unknown,anatom_site_general_anterior torso,anatom_site_general_head/neck,anatom_site_general_lower extremity,anatom_site_general_posterior torso,anatom_site_general_upper extremity,anatom_site_general_nan,tbp_tile_type_3D: XP,tbp_tile_type_3D: white
0,ISIC_0193705,0,IP_4156836,40,0,2.78,15.531807,11.774579,32.906984,28.568145,...,0,0,0,0,0,0,1,0,1,0
1,ISIC_0193743,0,IP_4540594,65,1,3.42,17.387463,13.946408,29.634201,28.488646,...,0,0,0,0,0,0,1,0,1,0
2,ISIC_0193803,0,IP_9726832,85,1,4.10,21.996120,16.950750,33.752600,33.537390,...,0,0,0,0,0,1,0,0,1,0
3,ISIC_0193817,0,IP_9067016,65,1,3.14,19.182800,16.012870,27.071170,26.853360,...,1,0,1,0,0,0,0,0,0,1
4,ISIC_0193830,0,IP_1930850,60,1,7.20,17.974908,10.425062,21.599385,21.324916,...,0,0,0,0,0,1,0,0,1,0


In [ ]:
import h5py
from sklearn.model_selection import train_test_split
# Load the metadata
# metadata_df = pd.read_csv('train-metadata.csv')
train_hd5 = "train-image.hdf5"

# Load the HDF5 file containing the images
hdf5_file = h5py.File(train_hd5, 'r')

# Get the image IDs (isic_id)
image_ids = list(hdf5_file.keys())

# Ensure the metadata is consistent with the image IDs
final_df_train = final_df_train[final_df_train['isic_id'].isin(image_ids)]

# Split the dataset into 80% for training and 20% for validation
train_ids, val_ids = train_test_split(final_df_train, test_size=0.2, random_state=42, stratify=final_df_train['target'])  # Stratified split to maintain class balance


In [ ]:
final_df_train["target"].value_counts()

In [ ]:
val_ids.target.value_counts()

In [ ]:
print(f'train size: {train_ids.shape}, val size: {val_ids.shape}')

In [2]:
import sys
sys.path.append('../src')
sys.path.append('../Models')
from CNN_model import EfficientNetTrainer
# from src import dataloader
from dataloader import ISICDataset
from torch.utils.data import Dataset, DataLoader
from augmentation import augmentation_pipeline


In [3]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

image_size = 224  

transforms_train = A.Compose([
    A.Transpose(p=0.5),
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, p=0.75),
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=0.7),
    A.OneOf([
        A.OpticalDistortion(distort_limit=1.0),
        A.GridDistortion(num_steps=5, distort_limit=1.),
        A.ElasticTransform(alpha=3),
    ], p=0.7),
    A.CLAHE(clip_limit=4.0, p=0.7),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
    A.Resize(image_size, image_size),
    A.CoarseDropout(max_holes=1, max_height=int(image_size * 0.375), max_width=int(image_size * 0.375), fill_value=0, p=0.7),
    ToTensorV2(),
    
])
transformations_valid = A.Compose([
    A.Resize(image_size,image_size ),
    A.Normalize(
            mean=[0.4815, 0.4578, 0.4082], 
            std=[0.2686, 0.2613, 0.2758], 
            max_pixel_value=255.0,
            p=1.0
        ),
    ToTensorV2(),
    ], p=1.)

def create_dataloader(hd5file, csvfile, transform, batch_size=32):
    dataset = ISICDataset(hd5file,csvfile, transform)
    return DataLoader(dataset, batch_size=batch_size, shuffle= True)

train_hd5 = "E:/Mine/Skin_cancer_detection/isic-2024-challenge/train-image.hdf5"
train_loader = create_dataloader(final_df_train,train_hd5, transforms_train)
# val_loader = create_dataloader(val_ids,train_hd5, transformations_valid)


c:\Users\Administrator\miniconda3\envs\skin_cancer_env\Lib\site-packages\pydantic\main.py:212: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [ ]:
train_loader.dataset.targets

In [ ]:
# trainer = EfficientNetTrainer(num_classes=2, learning_rate=1e-4)
# trainer.train(train_loader, val_loader, epochs=5)
# Assuming you have dataset, groups, and labels ready
trainer = EfficientNetTrainer(num_classes=10, learning_rate=1e-3)
trainer.stratified_k_fold_train(train_loader.dataset, train_loader.dataset.targets, k=5, epochs=5)

# trainer.save_model('efficientnet_best.pth')
# trainer.model

In [ ]:

# trainer.save_model('efficientnet_best.pth')